In [8]:
# carregar dataset rudementar

# ===========================================================
# COLETA DE POIs TURÍSTICOS EM REGIÕES DE ANGOLA (OSMNX)
# VERSÃO ROBUSTA (corrige colunas ausentes e CRS)
# ===========================================================

import osmnx as ox
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from pathlib import Path

# === Diretórios ===
DATA_DIR = Path("../data/raw")
DATA_DIR.mkdir(parents=True, exist_ok=True)

# === Locais Turísticos ===
locais_turisticos = [
    {"nome": "Quedas de Calandula", "local": "Malanje, Angola"},
    {"nome": "Miradouro da Lua", "local": "Luanda, Angola"},
    {"nome": "Parque da Kissama", "local": "Luanda, Angola"}
]

# === Configurações ===
BUFFER_RADIUS = 5000  # metros
todos_pois = []

# === Tags OSM de interesse ===
TAGS = {
    "tourism": ["hotel", "guest_house", "motel", "camp_site", "viewpoint"],
    "amenity": ["restaurant", "bar", "cafe"],
    "leisure": True,
    "highway": True,
    "waterway": True,
    "natural": True
}

# ===========================================================
# LOOP PRINCIPAL
# ===========================================================
for item in locais_turisticos:
    nome, local = item["nome"], item["local"]
    print(f"\n Coletando POIs em torno de {nome}...")

    try:
        # Obter coordenadas
        ponto = ox.geocode(f"{nome}, {local}")
        centro = Point(ponto[::-1])
        print(f"    Coordenadas: {ponto}")

        #  Criar buffer de busca
        area = gpd.GeoSeries([centro], crs="EPSG:4326").to_crs(3857).buffer(BUFFER_RADIUS).to_crs(4326).iloc[0]

        #  Buscar dados OSM
        pois = ox.features_from_polygon(area, TAGS).reset_index()

        #  Garantir colunas esperadas
        colunas_desejadas = ["osm_id", "name", "tourism", "amenity", "leisure",
                              "natural", "highway", "waterway", "geometry"]
        for col in colunas_desejadas:
            if col not in pois.columns:
                pois[col] = None

        pois = pois[colunas_desejadas]

        #  Adicionar metadados
        pois["ponto_turistico"] = nome
        pois["localidade"] = local
        pois["fonte_dados"] = "OpenStreetMap"

        # Corrigir cálculo de latitude e longitude
        pois_proj = pois.to_crs(3857)
        pois["latitude"] = pois_proj.geometry.centroid.to_crs(4326).y
        pois["longitude"] = pois_proj.geometry.centroid.to_crs(4326).x

        #  Salvar individualmente
        file_path = DATA_DIR / f"pois_{nome.replace(' ', '_').lower()}.geojson"
        pois.to_file(file_path, driver="GeoJSON")
        print(f"    Salvo: {file_path}")

        #  Mostrar amostra (somente colunas que existirem)
        colunas_para_mostrar = [c for c in ["name", "tourism", "amenity", "leisure", "natural", "latitude", "longitude"] if c in pois.columns]
        print("\n Amostra de POIs:")
        display(pois.head(10)[colunas_para_mostrar].fillna("—"))

        #  Resumo das categorias encontradas
        print("\n Categorias encontradas:")
        for col in ["tourism", "amenity", "leisure", "natural"]:
            if col in pois.columns:
                valores = pois[col].dropna().unique().tolist()
                if valores:
                    print(f"   - {col}: {valores[:5]}")

        todos_pois.append(pois)

    except Exception as e:
        print(f" Erro ao processar {nome}: {e}")

# ===========================================================
# COMBINAR RESULTADOS
# ===========================================================
if todos_pois:
    pois_final = gpd.GeoDataFrame(pd.concat(todos_pois, ignore_index=True), crs="EPSG:4326")
    arquivo_final = DATA_DIR / "pontos_turisticos_angola.geojson"
    pois_final.to_file(arquivo_final, driver="GeoJSON")
    print(f"\n Arquivo final combinado salvo em: {arquivo_final}")
else:
    print("\n Nenhum POI foi coletado.")



 Coletando POIs em torno de Quedas de Calandula...
    Coordenadas: (-9.0740668, 16.000316)
    Salvo: ..\data\raw\pois_quedas_de_calandula.geojson

 Amostra de POIs:


,name,tourism,amenity,leisure,natural,latitude,longitude
0,Quedas de Calandula,—,—,—,—,-9.074067,16.000316
1,—,—,—,—,—,-9.074009,15.998896
2,—,—,—,—,—,-9.074206,16.002487
3,—,camp_site,—,—,—,-9.077999,16.002056
4,Estrada de Calandula,—,—,—,—,-9.111147,15.970919
5,Ligação Quedas Calandula,—,—,—,—,-9.079526,15.977299
6,Lucala,—,—,—,—,-9.072398,16.002770
7,Parque das Quedas,picnic_site,—,park,—,-9.074234,15.999365
8,—,—,—,—,—,-9.074260,15.999257
9,—,—,—,—,—,-9.087594,16.013278



 Categorias encontradas:
   - tourism: ['camp_site', 'picnic_site', 'hotel']
   - amenity: ['restaurant']
   - leisure: ['park', 'swimming_pool']
   - natural: ['cliff', 'water']

 Coletando POIs em torno de Miradouro da Lua...
    Coordenadas: (-9.2211748, 13.0898298)
    Salvo: ..\data\raw\pois_miradouro_da_lua.geojson

 Amostra de POIs:


,name,tourism,amenity,leisure,natural,latitude,longitude
0,—,—,—,—,cliff,-9.262782,13.100828
1,—,—,—,—,cliff,-9.262581,13.100718
2,—,—,—,—,tree,-9.215676,13.090128
3,—,—,—,—,tree,-9.219566,13.090424
4,—,—,—,—,—,-9.220982,13.090689
5,—,—,—,—,tree,-9.221005,13.090521
6,—,—,—,—,tree,-9.220908,13.090492
7,—,—,—,—,tree,-9.221052,13.090077
8,—,—,—,—,tree,-9.221129,13.090214
9,—,—,—,—,tree,-9.243689,13.110539



 Categorias encontradas:
   - leisure: ['resort', 'swimming_pool']
   - natural: ['cliff', 'tree', 'coastline']

 Coletando POIs em torno de Parque da Kissama...
    Coordenadas: (-9.3611652, 13.1764564)
    Salvo: ..\data\raw\pois_parque_da_kissama.geojson

 Amostra de POIs:


,name,tourism,amenity,leisure,natural,latitude,longitude
0,Imbondeiro,—,restaurant,—,—,-9.337863,13.157969
1,—,—,restaurant,—,—,-9.342203,13.153666
2,—,—,—,—,wood,-9.333692,13.164577
3,—,—,—,—,water,-9.291340,13.185799
4,—,—,—,—,wood,-9.325136,13.168571
5,—,—,—,—,wood,-9.323814,13.192610
6,—,—,—,—,wood,-9.314491,13.178691
7,—,—,—,—,wood,-9.295153,13.173146
8,—,—,—,—,water,-9.356115,13.176297
9,—,—,—,—,coastline,-9.573579,13.203837



 Categorias encontradas:
   - amenity: ['restaurant']
   - leisure: ['resort', 'golf_course', 'pitch']
   - natural: ['wood', 'water', 'coastline', 'sand']

 Arquivo final combinado salvo em: ..\data\raw\pontos_turisticos_angola.geojson


In [ ]:
# Limpeza e normalização


In [ ]:
# Gerar variáveis espaciais


In [ ]:
# Cruzar camadas (join espacial)


In [ ]:
# Salvar dataset final


In [ ]:
# Entrega esperada: model_inputs.csv